# 모델을 만들기 위한 절차

1. 2개 이상의 데이터가 들어오면 자동으로 처리할 수 있게 해주는 모델 개발
2. 현재는 4개의 데이터를 직접 전처리해 한개의 데이터를 추출할 수 있게 해놓음
3. 기온과 습도 풍향과 풍속은 그날 바로 알 수 있으므로, 이 4개 데이터를 넣으면 그 지역 현재 시간대의 미세먼지 등급을 알 수 있게 하는 모델

결론적으로 처리할 건  
(이렇게 하려면 실시간으로 데이터를 크롤링 해와 데이터를 쉽게 다룰 수 있도록 바꾸는 과정이 필요함. -> 현재 불가)  
(그냥 과거 데이터를 그냥 두고, 입력을 4개하면 그 지역 미세먼지 등급을 예측하는 모델을 생각해보자.)  
input : 입력하세요 -> 기온, 습도, 풍향, 풍속, 위도(거의 비슷한 위치이므로 생략), 경도(거의 비슷한 위치이므로 생략)  
처리중입니다. -> 결과 : **지역 미세먼지 등급 - 좋음



다음에 해야할 작업 - >
1. 위도, 경도 데이터를 대안동을 뺀 나머지 데이터에 다 넣어주고,
2. 위도, 경도, 풍향, 풍속, 기온, 습도 6가지 데이터의 특징을 찾아보고 (없을 확률이 높다)  

-> 한개지역의 데이터씩 뽑아서 <올라가거나 내려가는 구간>에서는 회귀분석을 통해, 나온 예측값들을 조화평균내서 대안동에 대입하고, 거의 일정한 구간에서는 그 값들의 평균을 내 대안동에 대입해서 pm25를 예측하는 알고리즘을 만들어서 만들고, 그 값들을 SVM, KNN 등등에 대입해 4가지 등급을 예측하는 알고리즘을 만들자 


-> 올라가거나 내려가는 구간을 판별하는 방식은 경사하강법? 미분을 적용시켜 속도가 변하는 구간이 발생하는 조건을 넣어서 생기는 구간을 만들면 되지 않을까 생각한다.

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta, datetime
from matplotlib import font_manager, rc
import warnings

# plt parameter 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['figure.figsize'] = [25,15]
plt.rcParams['font.size'] = 25

# 경고메세지 끄기
warnings.filterwarnings(action='ignore')

In [2]:
# outliar 제거 함수 (IQR 설정)
def remove_out(dataframe0, dataframe2, dataframe3, dataframe4, dataframe5, remove_col) :
    dff = [dataframe0, dataframe2, dataframe3, dataframe4, dataframe5]
    for i, dataframe in enumerate(dff) :
        for k in remove_col :
            level_1q = dataframe[k].quantile(0.25)
            level_3q = dataframe[k].quantile(0.75)
            IQR = level_3q - level_1q
            rev_range = 10
            
            outliar_h = dataframe[k] >= level_3q + (rev_range * IQR)
            outliar_l = dataframe[k] <= level_1q - (rev_range * IQR)

            a=dataframe[outliar_h].index
            b=dataframe[outliar_l].index
            
            print(dataframe.name,":",[a,b],"\n")
            
            dataframe.drop(a, inplace=True)
            dataframe.drop(b, inplace=True)

# 아림의 미세먼지 농도가 매우나쁨 이상일 때, airKorea의 미세먼지 농도의 평가
def air_quality_label(pm25):
    if pm25 <= 15:
        return '좋음'
    elif pm25 <= 35:
        return '보통'
    elif pm25 <= 75:
        return '나쁨'
    else:
        return '매우 나쁨'

In [3]:
# arim_data 1.01~1.15 Hour
OA200_2 = pd.read_csv('./OA200_2/OA2002_01.01~01.15.csv')
OA200_2['reg_date'] = OA200_2.reg_date.apply(pd.to_datetime)

OA200_1 = pd.read_csv('./OA200_1/OA2001_01.01~01.15.csv')
OA200_1['reg_date'] = OA200_1.reg_date.apply(pd.to_datetime)

# airKorea_data 1.16~1.31 Hour, 상봉동 : 1, 상대동 : 2, 대안동 : 3
airKorea_data_1 = pd.read_csv('./AirKorea_상봉동/AirKorea_01.01~01.15.csv')
airKorea_data_1['date'] = airKorea_data_1.date.apply(pd.to_datetime)

airKorea_data_2 = pd.read_csv('./AirKorea_상대동/AirKorea_01.01~01.15.csv')
airKorea_data_2['date'] = airKorea_data_2.date.apply(pd.to_datetime)

airKorea_data_3 = pd.read_csv('./AirKorea_대안동/AirKorea_01.01~01.15.csv')
airKorea_data_3['date'] = airKorea_data_3.date.apply(pd.to_datetime)

# 이름 설정
OA200_1.name = '학교'
OA200_2.name = '진주성'
airKorea_data_1.name = '상봉동'
airKorea_data_2.name = '상대동'
airKorea_data_3.name = '대안동'

# outliar 제거
remove_out(OA200_1, OA200_2, airKorea_data_1, airKorea_data_2, airKorea_data_3, ['pm25'])

# 1시간 단위로 측정 나누기
OA200_2_avg = OA200_2.groupby([pd.Grouper(key='reg_date', freq='H')]).mean()
OA200_1_avg = OA200_1.groupby([pd.Grouper(key='reg_date', freq='H')]).mean()
airKorea_data_1_avg = airKorea_data_1.groupby([pd.Grouper(key='date', freq='H')]).mean()
airKorea_data_2_avg = airKorea_data_2.groupby([pd.Grouper(key='date', freq='H')]).mean()
airKorea_data_3_avg = airKorea_data_3.groupby([pd.Grouper(key='date', freq='H')]).mean()

# arim 데이터 수정
OA200_1_avg_76 = OA200_1_avg.loc[OA200_1_avg.loc[OA200_1_avg.pm25 >= 76].index]
OA200_2_avg_76 = OA200_2_avg.loc[OA200_2_avg.loc[OA200_2_avg.pm25 >= 76].index]
OA200_1_avg_76.pm25 -= 4.49
OA200_2_avg_76.pm25 -= 4.49

# arim 데이터 수정 후 대입
OA200_1_avg.loc[OA200_1_avg_76.index, 'pm25'] = OA200_1_avg_76['pm25']
OA200_2_avg.loc[OA200_2_avg_76.index, 'pm25'] = OA200_2_avg_76['pm25']

# 좋음, 보통, 나쁨, 매우나쁨 카데고리 분류
OA200_1_avg['air_quality_label'] = OA200_1_avg['pm25'].apply(lambda x: air_quality_label(x))
OA200_2_avg['air_quality_label'] = OA200_2_avg['pm25'].apply(lambda x: air_quality_label(x))
airKorea_data_1_avg['air_quality_label'] = airKorea_data_1_avg['pm25'].apply(lambda x: air_quality_label(x))
airKorea_data_2_avg['air_quality_label'] = airKorea_data_2_avg['pm25'].apply(lambda x: air_quality_label(x))
airKorea_data_3_avg['air_quality_label'] = airKorea_data_3_avg['pm25'].apply(lambda x: air_quality_label(x))

# arim 데이터 풍향, 풍속 데이터 삽입
Sangbong = pd.read_csv("./weather/상봉동/2023-01.csv", encoding='cp949')
Sangbong['일시'] = Sangbong['일시'].apply(pd.to_datetime)
Sangbong_avg = Sangbong.groupby([pd.Grouper(key='일시', freq='H')]).mean()
OA200_2_avg['wind_dir'] = Sangbong_avg['풍향(16방위)']
OA200_2_avg['wind_speed'] = Sangbong_avg['풍속(m/s)']

Jungchon = pd.read_csv("./weather/정촌면(학교근처)/2023-01.csv", encoding='cp949')
Jungchon['일시'] = Jungchon['일시'].apply(pd.to_datetime)
Jungchon_avg = Jungchon.groupby([pd.Grouper(key='일시', freq='H')]).mean()
OA200_1_avg['wind_dir'] = Jungchon_avg['풍향(16방위)']
OA200_1_avg['wind_speed'] = Jungchon_avg['풍속(m/s)']

# airKorea 데이터 온도, 습도, 풍향, 풍속 데이터 삽입
airKorea_data_1_avg['temp'] = Sangbong_avg['기온']
airKorea_data_1_avg['humi'] = Sangbong_avg['상대습도(%)']
airKorea_data_1_avg['wind_dir'] = Sangbong_avg['풍향(16방위)']
airKorea_data_1_avg['wind_speed'] = Sangbong_avg['풍속(m/s)']

Sangdae = pd.read_csv("./weather/상대동/2023-01.csv", encoding='cp949')
Sangdae['일시'] = Sangdae['일시'].apply(pd.to_datetime)
Sangdae_avg = Sangdae.groupby([pd.Grouper(key='일시', freq='H')]).mean()
airKorea_data_2_avg['temp'] = Sangdae_avg['기온']
airKorea_data_2_avg['humi'] = Sangdae_avg['상대습도(%)']
airKorea_data_2_avg['wind_dir'] = Sangdae_avg['풍향(16방위)']
airKorea_data_2_avg['wind_speed'] = Sangdae_avg['풍속(m/s)']

Daean = pd.read_csv("./weather/대안동/2023-01.csv", encoding='cp949')
Daean['일시'] = Daean['일시'].apply(pd.to_datetime)
Daean_avg = Daean.groupby([pd.Grouper(key='일시', freq='H')]).mean()
airKorea_data_3_avg['temp'] = Daean_avg['기온']
airKorea_data_3_avg['humi'] = Daean_avg['상대습도(%)']
airKorea_data_3_avg['wind_dir'] = Daean_avg['풍향(16방위)']
airKorea_data_3_avg['wind_speed'] = Daean_avg['풍속(m/s)']

# OA200 데이터의 필요없는 no 열 삭제
OA200_1_avg = OA200_1_avg.drop('no', axis=1)
OA200_2_avg = OA200_2_avg.drop('no', axis=1)

# 일단 데이터 셋 확인
display(OA200_1_avg.info())
display(OA200_2_avg.info())
display(airKorea_data_1_avg.info())
display(airKorea_data_2_avg.info())
display(airKorea_data_3_avg.info())

# 참고로 Nan 값을 가진 대안동의 데이터를 소거 해주는 작업이 필요!
airKorea_data_1_avg = airKorea_data_1_avg.dropna(subset=['pm25'], axis=0)
airKorea_data_2_avg = airKorea_data_2_avg.dropna(subset=['pm25'], axis=0)
airKorea_data_3_avg = airKorea_data_3_avg.dropna(subset=['pm25'], axis=0)
OA200_1_avg = OA200_1_avg.dropna(subset=['pm25'], axis=0)
OA200_2_avg = OA200_2_avg.dropna(subset=['pm25'], axis=0)

# 가장 작은 airKorea_data의 인덱스와 나머지 데이터들의 인덱스 같게 만들어주기
idx = set(airKorea_data_1_avg.index).intersection(
    set(airKorea_data_2_avg.index),
    set(airKorea_data_3_avg.index),
    set(OA200_1_avg.index),
    set(OA200_2_avg.index)
)
airKorea_data_1_avg = airKorea_data_1_avg.reindex(idx)
airKorea_data_2_avg = airKorea_data_2_avg.reindex(idx)
airKorea_data_3_avg = airKorea_data_3_avg.reindex(idx)
OA200_1_avg = OA200_1_avg.reindex(idx)
OA200_2_avg = OA200_2_avg.reindex(idx)

# 수정 후 데이터 셋 확인
print("###################################\n"*5)
print("수정후")

# 일단 데이터 셋 확인
display(OA200_1_avg.info())
display(OA200_2_avg.info())
display(airKorea_data_1_avg.info())
display(airKorea_data_2_avg.info())
display(airKorea_data_3_avg.info())

학교 : [Int64Index([], dtype='int64'), Int64Index([], dtype='int64')] 

진주성 : [Int64Index([], dtype='int64'), Int64Index([], dtype='int64')] 

상봉동 : [Int64Index([], dtype='int64'), Int64Index([], dtype='int64')] 

상대동 : [Int64Index([], dtype='int64'), Int64Index([], dtype='int64')] 

대안동 : [Int64Index([], dtype='int64'), Int64Index([], dtype='int64')] 

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 359 entries, 2023-01-01 01:00:00 to 2023-01-15 23:00:00
Freq: H
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pm25               350 non-null    float64
 1   pm10               350 non-null    float64
 2   temp               350 non-null    float64
 3   humi               350 non-null    float64
 4   air_quality_label  359 non-null    object 
 5   wind_dir           359 non-null    float64
 6   wind_speed         359 non-null    float64
dtypes: float64(6), object(1)
memory usage: 30.5+ KB


None

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 359 entries, 2023-01-01 01:00:00 to 2023-01-15 23:00:00
Freq: H
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pm25               350 non-null    float64
 1   pm10               350 non-null    float64
 2   temp               350 non-null    float64
 3   humi               350 non-null    float64
 4   air_quality_label  359 non-null    object 
 5   wind_dir           359 non-null    float64
 6   wind_speed         359 non-null    float64
dtypes: float64(6), object(1)
memory usage: 30.5+ KB


None

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 359 entries, 2023-01-01 01:00:00 to 2023-01-15 23:00:00
Freq: H
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pm10               347 non-null    float64
 1   pm25               319 non-null    float64
 2   air_quality_label  359 non-null    object 
 3   temp               359 non-null    float64
 4   humi               359 non-null    float64
 5   wind_dir           359 non-null    float64
 6   wind_speed         359 non-null    float64
dtypes: float64(6), object(1)
memory usage: 22.4+ KB


...

None

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 253 entries, 2023-01-14 06:00:00 to 2023-01-03 00:00:00
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pm10               253 non-null    float64
 1   pm25               253 non-null    float64
 2   air_quality_label  253 non-null    object 
 3   temp               253 non-null    float64
 4   humi               253 non-null    float64
 5   wind_dir           253 non-null    float64
 6   wind_speed         253 non-null    float64
dtypes: float64(6), object(1)
memory usage: 15.8+ KB


None

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 253 entries, 2023-01-14 06:00:00 to 2023-01-03 00:00:00
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pm10               123 non-null    float64
 1   pm25               253 non-null    float64
 2   air_quality_label  253 non-null    object 
 3   temp               253 non-null    float64
 4   humi               253 non-null    float64
 5   wind_dir           253 non-null    float64
 6   wind_speed         253 non-null    float64
dtypes: float64(6), object(1)
memory usage: 15.8+ KB


None